referenced: https://github.com/BojarLab/glycowork/blob/master/05_examples.ipynb

In [1]:
from glycowork.ml.inference import get_lectin_preds
from glycowork.ml.models import prep_model
from glycowork.motif.graph import glycan_to_nxGraph
from glycowork.glycan_data.loader import lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.metrics import mean_squared_error

### Load Data

In [2]:
gold_data = pd.read_csv('../data_cleaning/gold_data_iupac.csv')
gold_data.head()

,Glycan SMILE,Glycan IUPAC,Protein Sequence,concentration,fraction_bound
0,OC[C@@H](O1)[C@H](O)[C@H](O)[C@@H](O)[C@H]1-OC...,Gal(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,0.001,0.000000
1,OC[C@@H](O1)[C@@H](O)[C@H](O)[C@@H](O)[C@H]1-O...,Glc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,0.001,0.000154
2,OC[C@@H](O1)[C@@H](O)[C@H](O)[C@H](O)[C@H]1-OC...,Man(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,0.001,0.000082
3,OC[C@@H](O1)[C@H](O)[C@H](O)[C@@H](NC(=O)C)[C@...,GalNAc(α-Sp15,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,0.001,0.000290
4,OC[C@@H](O1)[C@H](O)[C@H](O)[C@@H](NC(=O)C)[C@...,GalNAc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,0.001,0.000000


### LectinOracle Predictions

LectinOracle Does not consider the concentration of the protein when making predictions. So, the size of the gold dataset gets reduced.

In [3]:
pred_df = gold_data[['Glycan IUPAC', 'Protein Sequence']].drop_duplicates().reset_index(drop=True)
pred_df = pred_df.rename(columns={"Glycan IUPAC": "glycan", "Protein Sequence": "protein"})
pred_df["rfu_pred"] = np.nan
print(len(pred_df))
pred_df.head()

31760


,glycan,protein,rfu_pred
0,Gal(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN
1,Glc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN
2,Man(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN
3,GalNAc(α-Sp15,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN
4,GalNAc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN


In [4]:
leor = prep_model("LectinOracle_flex", 1, trained = True)

Some of the glycan IUPACs are recognized by LectinOracle/SweetNet, and some are not. LectinOracle allows you to make predictions for a protein and a set of glycans, but if even one of the glycan is not recognized then it throws an error. So, we need to filter out the bad (not recognized) glycans.

In [5]:
bad_glycans = set()
for glycan in pred_df['glycan'].unique():
    try:
        glycan_to_nxGraph(glycan, libr=lib)
    except KeyError:
        bad_glycans.add(glycan)
print(f"Bad Glycans: {len(bad_glycans)} / {len(pred_df['glycan'].unique())}")

Bad Glycans: 164 / 611


In [6]:
valid_glycan_df = pred_df[~pred_df["glycan"].isin(bad_glycans)].reset_index(drop=True)
print(len(valid_glycan_df))
valid_glycan_df.head()

23232


,glycan,protein,rfu_pred
0,Gal(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN
1,Glc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN
2,Man(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN
3,GalNAc(α-Sp15,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN
4,GalNAc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,NaN


In [7]:
# get lectinoracle predictions
for protein in valid_glycan_df['protein'].unique():
    prot_indices = (valid_glycan_df['protein'] == protein)
    prot_df = valid_glycan_df[prot_indices]
    glycans = list(prot_df['glycan'])
    
    leor_preds_df = get_lectin_preds(protein, glycans, leor, flex = True)
    valid_glycan_df.loc[prot_indices, "rfu_pred"] = list(leor_preds_df['pred'])
valid_glycan_df

,glycan,protein,rfu_pred
0,Gal(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.564553
1,Glc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.489788
2,Man(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.440447
3,GalNAc(α-Sp15,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.410684
4,GalNAc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.391872
...,...,...,...
23227,Neu5Ac(α2-6)Gal(β1-4)GlcNAc(β1-3)Gal(β1-4)GlcN...,ADTIVAVELDSYPNTDIGDPNYPHIGIDIKSIRSKSTARWNMQTGK...,3.116201
23228,Neu5Ac(α2-3)Gal(β1-4)GlcNAc(β1-3)Gal(β1-4)GlcN...,ADTIVAVELDSYPNTDIGDPNYPHIGIDIKSIRSKSTARWNMQTGK...,3.146052
23229,Neu5Ac(α2-6)Gal(β1-4)GlcNAc(β1-3)Gal(β1-4)GlcN...,ADTIVAVELDSYPNTDIGDPNYPHIGIDIKSIRSKSTARWNMQTGK...,3.160019
23230,GlcNAc(β1-3)Fuc(α-Sp21,ADTIVAVELDSYPNTDIGDPNYPHIGIDIKSIRSKSTARWNMQTGK...,3.164777


We need to convery the LectinOracle output from a Z-score transformed RFU value to universal fraction bound (as detailed in [Atom-level Machine Learning of Protein-glycan Interactions and Cross-chiral Recognition in Glycobiology](https://www.biorxiv.org/content/10.1101/2025.01.21.633632v3.supplementary-material), Supplementary Materials, Fig. S6,)

In [8]:
def min_max_scale(data: list, min_val: float, max_val: float) -> list:
    clipped_data = np.clip(data, min_val, max_val)
    return (clipped_data - min_val) / (max_val - min_val) 

def rfu_to_f(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['f_pred'] = np.nan
    rfu_max = df['rfu_pred'].max()
    for protein in df['protein'].unique():
        prot_indices = (df['protein'] == protein)
        prot_df = df[prot_indices]
        preds = list(prot_df['rfu_pred'])
        
        # use kernel density estimation to find the minimum RFU threshold
        kernel = gaussian_kde(preds, bw_method=0.2)
        x_vals = np.linspace(min(preds) - 0.001, max(preds) + 0.001, 500)
        y_vals = kernel(x_vals)
        rfu_threshold = x_vals[np.argmax(y_vals)]
        
        # linearly transform the range [rfu_threshold, rfu_max] -> [0, 1]
        f = min_max_scale(preds, rfu_threshold, rfu_max)
        df.loc[prot_indices, 'f_pred'] = f
    return df

In [9]:
leor_pred_df = rfu_to_f(valid_glycan_df)
leor_pred_df

,glycan,protein,rfu_pred,f_pred
0,Gal(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.564553,0.000000
1,Glc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.489788,0.000000
2,Man(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.440447,0.000000
3,GalNAc(α-Sp15,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.410684,0.000000
4,GalNAc(α-Sp8,MRLVAKLLYLAVLAICGLGIHGALTHPRVTPPVYPSVSFNLTGADT...,-0.391872,0.000000
...,...,...,...,...
23227,Neu5Ac(α2-6)Gal(β1-4)GlcNAc(β1-3)Gal(β1-4)GlcN...,ADTIVAVELDSYPNTDIGDPNYPHIGIDIKSIRSKSTARWNMQTGK...,3.116201,0.976734
23228,Neu5Ac(α2-3)Gal(β1-4)GlcNAc(β1-3)Gal(β1-4)GlcN...,ADTIVAVELDSYPNTDIGDPNYPHIGIDIKSIRSKSTARWNMQTGK...,3.146052,0.985540
23229,Neu5Ac(α2-6)Gal(β1-4)GlcNAc(β1-3)Gal(β1-4)GlcN...,ADTIVAVELDSYPNTDIGDPNYPHIGIDIKSIRSKSTARWNMQTGK...,3.160019,0.989661
23230,GlcNAc(β1-3)Fuc(α-Sp21,ADTIVAVELDSYPNTDIGDPNYPHIGIDIKSIRSKSTARWNMQTGK...,3.164777,0.991064


### Compute MSE

LectinOracle takes as input (glycan, protein) pairs, while the gold data takes as input (glycan, protein, concentration) triplets. So, when comparing LectinOracle to the gold data, we must somehow map a single LectinOracle prediction to multiple true predictions. To calculate the MSE, we try 3 different approaches:
1. Map the LectinOracle prediction to the fraction bound of EVERY concentration (full_preds)
2. Map the LectinOracle prediction to the average fraction bound of the different concentrations (avg_preds)
3. Map the LectinOracle prediction to the fraction bound that is closest to the predicted value. This serves as the lowest possible MSE LectinOracle can achieve (closest_preds)

In [10]:
# since we access rows of the gold_data based on glycan protein pairs, we reindex for efficiency
reindexed_gold_data = gold_data.set_index(['Glycan IUPAC', 'Protein Sequence'])
reindexed_gold_data.sort_index(inplace=True)

full_preds = {"pred": [], "true": []}
avg_preds = {"pred": list(leor_pred_df['f_pred']), "true": []}
closest_preds = {"pred": list(leor_pred_df['f_pred']), "true": []}

for i, row in leor_pred_df.iterrows():
    protein = row['protein']
    glycan = row['glycan']
    f_pred = row['f_pred']
    
    # for each glycan protein pair lectinoracle makes a prediction for, the gold_data contains multiple true values for different concentrations
    true_df = reindexed_gold_data.loc[(glycan, protein)]
    true_fs = true_df['fraction_bound'].values
    
    full_preds['true'] += list(true_fs)
    full_preds['pred'] += [f_pred for _ in true_fs]
    
    avg_f = true_fs.mean()
    avg_preds['true'].append(avg_f)
    
    # gets the true fraction bound that is closest to the predicted value
    closest_f = (abs(true_fs - f_pred)).min() + f_pred
    closest_preds['true'].append(closest_f)
    
full_mse = mean_squared_error(full_preds['true'], full_preds['pred'])
avg_mse = mean_squared_error(avg_preds['true'], avg_preds['pred'])
closest_mse = mean_squared_error(closest_preds['true'], closest_preds['pred'])
print(f'Full: {full_mse}, Average: {avg_mse}, Closest: {closest_mse}')

Full: 0.02311987643343187, Average: 0.019332950155647233, Closest: 0.01666273097406949
